In [9]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Chest_X_Ray-2\ChestXRay2017\chest_xray\train'

# Initialize empty lists for images and labels
X_train = []
Y_train = []

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Loop through subdirectories (classes)
for class_folder in os.listdir(directory_path):
    class_path = os.path.join(directory_path, class_folder)
    class_label = class_folder
    
    # Encode class labels
    encoded_label = label_encoder.fit_transform([class_label])[0]

    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpeg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(encoded_label)

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (5232, 32, 32, 3)
Y_train shape: (5232,)


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical format with 23 classes
Y_train = to_categorical(Y_train,2)

# Perform train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.4, random_state=42)

# Check the shapes
print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_test shape: {Y_test.shape}')

X_train shape: (3139, 32, 32, 3)
Y_train shape: (3139, 2)
X_test shape: (2093, 32, 32, 3)
Y_test shape: (2093, 2)


In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))  # Change the number of units and activation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_split = .2)

Epoch 1/100
40/40 [==============================] - 19s 117ms/step - loss: 0.0724 - accuracy: 0.9761 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
40/40 [==============================] - 3s 61ms/step - loss: 1.7308e-05 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/100
40/40 [==============================] - 2s 55ms/step - loss: 1.0988e-05 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/100
40/40 [==============================] - 2s 45ms/step - loss: 6.3786e-06 - accuracy: 1.0000 - val_loss: 5.1272e-04 - val_accuracy: 1.0000
Epoch 5/100
40/40 [==============================] - 2s 50ms/step - loss: 5.2506e-06 - accuracy: 1.0000 - val_loss: 1.8065e-04 - val_accuracy: 1.0000
Epoch 6/100
40/40 [==============================] - 2s 50ms/step - loss: 3.7554e-06 - accuracy: 1.0000 - val_loss: 7.3519e-05 - val_accuracy: 1.0000
Epoch 7/100
40/40 [==============================] - 2s 44ms/step - loss: 3.5048e-06 - accuracy: 1.0000 - val_

40/40 [==============================] - 2s 60ms/step - loss: 1.4409e-07 - accuracy: 1.0000 - val_loss: 1.6458e-07 - val_accuracy: 1.0000
Epoch 56/100
40/40 [==============================] - 2s 55ms/step - loss: 1.2002e-07 - accuracy: 1.0000 - val_loss: 1.5945e-07 - val_accuracy: 1.0000
Epoch 57/100
40/40 [==============================] - 3s 65ms/step - loss: 1.0876e-07 - accuracy: 1.0000 - val_loss: 1.5736e-07 - val_accuracy: 1.0000
Epoch 58/100
40/40 [==============================] - 2s 59ms/step - loss: 1.1926e-07 - accuracy: 1.0000 - val_loss: 1.5243e-07 - val_accuracy: 1.0000
Epoch 59/100
40/40 [==============================] - 2s 58ms/step - loss: 1.0321e-07 - accuracy: 1.0000 - val_loss: 1.3971e-07 - val_accuracy: 1.0000
Epoch 60/100
40/40 [==============================] - 2s 61ms/step - loss: 9.5899e-08 - accuracy: 1.0000 - val_loss: 1.3288e-07 - val_accuracy: 1.0000
Epoch 61/100
40/40 [==============================] - 2s 55ms/step - loss: 2.0114e-07 - accuracy: 1.0000 - 

In [12]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
batch_size = 64
import tensorflow as tf

# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = model.predict(test_dataset)

Y_test = np.argmax(Y_test , axis=1)
# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


33/33 [==============================] - 2s 16ms/step
Test Loss: 9.380604382158708e-08
Test Accuracy: 1.0
Accuracy: 1.0
F1 Score: 1.0


In [ ]:
from tensorflow import keras
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\limuc_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(2, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_split = .2)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = new_model.predict(test_dataset)

Y_test = np.argmax(Y_test , axis=1)
# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
